In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.model_selection import KFold
from utilities import cal_score

In [2]:
stack_idx = '02'
models = '1,3-5'

### Read CV predictions and test

In [3]:
def parse_models(exp):
    exp_split = exp.split(',')
    idx_models = []
    for e in exp_split:
        if '-' in e:
            n0, n1 = e.split('-')
            idx_models.extend(list(range(int(n0), int(n1)+1, 1)))
        else:
            idx_models.append(int(e))
    return idx_models

In [4]:
idx_models = parse_models(models)

In [5]:
files_in_output = [f for f in os.listdir('output/') if os.path.isfile('output/'+f)]
files_cv = [[f for f in files_in_output if 'model-%02d-' % idx in f and 'cv' in f][0] for idx in idx_models]
files_test_one = [[f for f in files_in_output if 'model-%02d-' % idx in f and 'test-one' in f][0] \
                  for idx in idx_models]
files_test_kf = [[f for f in files_in_output if 'model-%02d-' % idx in f and 'test-kfold' in f][0] \
                 for idx in idx_models]

In [6]:
files_test_kf

['model-01-lgb-test-kfold.csv',
 'model-03-lgb-feats-selection-test-kfold.csv',
 'model-04-lgb-PCA-test-kfold.csv',
 'model-05-lgb-wo-per-area-test-kfold.csv']

In [7]:
cv = None
test_one = None
test_kf = None

for i, f in enumerate(files_cv):
    df = pd.read_csv('output/'+f)
    if cv is None:
        cv = df[['building_id','total_price_predict']].copy()
    else:
        cv = pd.merge(cv, df[['building_id','total_price_predict']], on='building_id')
    cv = cv.rename(columns = {'total_price_predict':'pred_{}'.format(idx_models[i])})
    cv[f'log_pred_{idx_models[i]}'] = np.log1p(cv[f'pred_{idx_models[i]}'])
cv = pd.merge(cv, df[['building_id','total_price']], on='building_id')
cv['log_total_price'] = np.log1p(cv['total_price'])

for i, f in enumerate(files_test_one):
    df = pd.read_csv('output/'+f)
    if test_one is None:
        test_one = df[['building_id','total_price']].copy()
    else:
        test_one = pd.merge(test_one, df[['building_id','total_price']], on='building_id')
    test_one = test_one.rename(columns = {'total_price':'pred_{}'.format(idx_models[i])})
    test_one[f'log_pred_{idx_models[i]}'] = np.log1p(test_one[f'pred_{idx_models[i]}'])

In [8]:
test_one[['building_id','pred_5']].rename(columns={'building_id':'id','pred_5':'target'})\
    .to_csv('test_5.csv',index=False)

In [9]:
cv.head()

,building_id,pred_1,log_pred_1,pred_3,log_pred_3,pred_4,log_pred_4,pred_5,log_pred_5,total_price,log_total_price
0,jre1pJhcQj91Kdky,1.255350e+07,16.345510,1.255981e+07,16.346013,1.288381e+07,16.371482,1.282213e+07,16.366683,14215011.0,16.469809
1,jcbuA8q3KPH9SzpS,9.117771e+06,16.025736,9.675385e+06,16.085096,7.095057e+06,15.774909,9.570172e+06,16.074162,7642884.5,15.849286
2,EqWJpHmkXPyfHkB8,4.491672e+06,15.317736,4.511709e+06,15.322187,4.586149e+06,15.338551,4.687745e+06,15.360462,4807053.0,15.385595
3,ihrMWGM8Nq99Uvu6,6.918995e+06,15.749781,6.878882e+06,15.743967,6.906435e+06,15.747964,6.981481e+06,15.758772,7493026.5,15.829483
4,jRhwzoVMFLFzxAX1,2.750253e+06,14.827204,2.754684e+06,14.828814,2.725200e+06,14.818053,2.838090e+06,14.858642,2846855.8,14.861726


In [10]:
X = cv[['log_pred_{}'.format(idx) for idx in idx_models]]
y = cv['log_total_price']

reg = LassoCV(alphas=[0]+list(np.logspace(-4, 3, 7)), max_iter=100000, tol=1e-6, n_jobs=-1)
reg.fit(X, y)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:474: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:1239: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:4

LassoCV(alphas=[0, 0.0001, 0.0014677992676220704, 0.021544346900318846, 0.31622776601683794, 4.641588833612782, 68.12920690579622, 1000.0],
    copy_X=True, cv='warn', eps=0.001, fit_intercept=True, max_iter=100000,
    n_alphas=100, n_jobs=-1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=1e-06,
    verbose=False)

In [11]:
print(reg.alpha_)
print(reg.mse_path_)
print(reg.coef_, reg.intercept_)

0.0
[[1.3539862  1.37718519 1.34124855]
 [1.3539862  1.37718519 1.34124855]
 [1.3539862  1.37718519 1.34124855]
 [0.11307747 0.11793192 0.10999679]
 [0.03660905 0.03868912 0.03686091]
 [0.03621081 0.03822249 0.03660558]
 [0.0362071  0.03821452 0.03661186]
 [0.0362069  0.03821406 0.03661245]]
[0.11829519 0.50525191 0.06704509 0.31655351] -0.1101545626516689


In [12]:
for a in [0]+list(np.logspace(-4, 3, 7)):
    reg_single = Lasso(alpha=a, max_iter=100000, tol=1e-6)
    reg_single.fit(X, y)
    print(reg_single.coef_, reg.intercept_)
    print(reg_single.score(X,y))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[0.11829519 0.50525191 0.06704509 0.31655351] -0.1101545626516689
0.9727605713710098
[0.11954385 0.50449966 0.06680784 0.31621256] -0.1101545626516689
0.9727605647988661
[0.12194848 0.50678326 0.06451921 0.31271114] -0.1101545626516689
0.9727592897718434
[0.15724185 0.54030338 0.03092693 0.26131758] -0.1101545626516689
0.9724850931568716
[0.17526768 0.58669113 0.         0.        ] -0.1101545626516689
0.9162849597637844
[0. 0. 0. 0.] -0.1101545626516689
0.0
[0. 0. 0. 0.] -0.1101545626516689
0.0
[0. 0. 0. 0.] -0.1101545626516689
0.0


In [13]:
alphas = [0, 0.0001, 0.0002, 0.0005, 0.0008, 0.001, 0.002, 0.005, 0.008, 0.01, 0.02]
gsearch = {}

folds = KFold(n_splits=3, shuffle=True, random_state=1208)
for i_fold, (itrain, ival) in enumerate(folds.split(X)): # kfold
    print('==== Fold', i_fold+1, '====')
    
    # split train, val
    X_train = X.iloc[itrain]
    X_val = X.iloc[ival]
    y_train = y.iloc[itrain]
    y_val = y.iloc[ival]
    
    # random sample - grid search
    for a in alphas:
        if a == 0:
            reg_single = LinearRegression()
        else:
            reg_single = Lasso(alpha=a, max_iter=100000, tol=1e-6)
        reg_single.fit(X_train, y_train)
        
        y_pred = reg_single.predict(X_val)
        y_pred_final = np.expm1(y_pred)
        y_true_final = np.expm1(y_val)
        score = cal_score(y_true_final, y_pred_final)

        print('alpha, score:', a, score)
        gsearch[a] = gsearch.get(a,[]) + [score]

results = [[key, np.mean(value), value] for key, value, in gsearch.items()]
results.sort(key= lambda x: x[1], reverse=True)
for item in results:
    print(item)

==== Fold 1 ====
alpha, score: 0 5905.872039229822
alpha, score: 0.0001 5904.872042301729
alpha, score: 0.0002 5904.872045147653
alpha, score: 0.0005 5900.8720532992875
alpha, score: 0.0008 5898.872060963754
alpha, score: 0.001 5900.872065798205
alpha, score: 0.002 5898.872086026499
alpha, score: 0.005 5898.872108674413
alpha, score: 0.008 5894.872050063835
alpha, score: 0.01 5895.87194951383
alpha, score: 0.02 5862.8708548446275
==== Fold 2 ====
alpha, score: 0 5856.873518651702
alpha, score: 0.0001 5855.873522694555
alpha, score: 0.0002 5858.8735268822375
alpha, score: 0.0005 5858.873539125245
alpha, score: 0.0008 5856.873550567549
alpha, score: 0.001 5857.873557895186
alpha, score: 0.002 5858.873590629856
alpha, score: 0.005 5867.873637046685
alpha, score: 0.008 5871.873605774259
alpha, score: 0.01 5864.87352685283
alpha, score: 0.02 5854.872573943387
==== Fold 3 ====
alpha, score: 0 5922.876755924785
alpha, score: 0.0001 5923.876758951287
alpha, score: 0.0002 5922.876761442709
alph

In [14]:
alpha_set = results[0][0]
print(alpha_set)
if alpha_set == 0:
    reg = LinearRegression()
else:
    reg = Lasso(alpha=alpha_set, max_iter=1000000, tol=1e-6)
reg.fit(X, y)

0.008


Lasso(alpha=0.008, copy_X=True, fit_intercept=True, max_iter=1000000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=1e-06, warm_start=False)

In [15]:
print(reg.coef_, reg.intercept_)

[0.13343172 0.51768951 0.05358947 0.29598949] -0.010131829524533842


### Calculate cv score

In [16]:
cv_pred_final = np.zeros(X.shape[0])
for i, col in enumerate(X):
    cv_pred_final = cv_pred_final + X[col] * reg.coef_[i]
cv_pred_final = cv_pred_final + reg.intercept_
cv_pred_final = np.expm1(cv_pred_final)
cv_true_final = np.expm1(y)

In [17]:
pd.DataFrame({'a':cv_true_final,'b':cv_pred_final}).head()

,a,b
0,14215011.0,1.266993e+07
1,7642884.5,9.420953e+06
2,4807053.0,4.567129e+06
3,7493026.5,6.922106e+06
4,2846855.8,2.777611e+06


In [18]:
cal_score(cv_true_final, cv_pred_final)

5894.874161888201

### Compute submission

In [19]:
test_pred_final = pd.DataFrame({'building_id': test_one['building_id'], 'total_price': np.zeros(test_one.shape[0])})
for i, idx in enumerate(idx_models):
    test_pred_final['total_price'] = test_pred_final['total_price'] + test_one[f'log_pred_{idx}'] * reg.coef_[i]
test_pred_final['total_price'] = test_pred_final['total_price'] + reg.intercept_
test_pred_final['total_price'] = np.expm1(test_pred_final['total_price'])
test_pred_final['total_price'] = np.clip(test_pred_final['total_price'], 0, None)

test_pred_final.to_csv('output/stack_{}_{}.csv'.format(stack_idx, models), index=False)

In [23]:
a= pd.read_csv('output/model-03-lgb-feats-selection-cv.csv')
b= pd.read_csv('output/model-03-lgb-feats-selection-test-one.csv')

In [25]:
a.rename(columns={'building_id':'id','total_price_predict':'target'}).to_csv('opt-pred3.csv',index=False)
b.rename(columns={'building_id':'id','total_price':'target'}).to_csv('opt-test3.csv',index=False)